In [4]:
!pip install requests

In [5]:
import requests
import os
import tarfile

# COCO 데이터셋의 다운로드 링크
url = "http://images.cocodataset.org/zips/val2017.zip"
zip_file = "val2017.zip"
extracted_folder = "val2017"

# COCO 데이터셋 다운로드
response = requests.get(url, stream=True)
response.raise_for_status()

# 다운로드한 데이터 저장
with open(zip_file, "wb") as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

In [9]:
import zipfile

# 압축 해제된 파일이 저장될 경로
extracted_dir = "./"

# zip 압축 해제
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

In [10]:
import os

from minio import Minio

minio_client = Minio(
    "localhost:57100",
    access_key="mlops",
    secret_key="mlops123",
    secure=False,
)

# 업로드할 폴더 경로
folder_path = './val2017'

# MinIO 버킷 정보 
bucket_name = 'raw-data'

# 폴더 내 파일 업로드
for root, dirs, files in os.walk(folder_path):
    for file_name in files:
        file_path = os.path.join(root, file_name)
        object_name = os.path.relpath(file_path, folder_path)
        minio_client.fput_object(bucket_name, f"coco/val2017/{object_name}", file_path)

In [11]:
# 업로드한 파일 삭제
from typing import List 

from minio.deleteobjects import DeleteError, DeleteObject

prefix="coco"

minio_objects = minio_client.list_objects(
    bucket_name=bucket_name,
    prefix=prefix,
    recursive=True,
)

delete_objects = [DeleteObject(obj.object_name) for obj in minio_objects]

errors: List[DeleteError] = minio_client.remove_objects(
    bucket_name,
    delete_objects,
)
error = next(errors, None)
if error is not None:
    print("Bad news arrived")

KeyboardInterrupt: 